In [120]:
from gurobipy import GRB
import gurobipy as gb
from gurobipy import *
import numpy as np

# Question 1

# a

In [121]:
isVariablePricing = True

In [122]:
# Linear price response functions (intercept, slope)
response = [[35234.5457855123, 45.8964497063843], [37790.2408321369, 8.22779417263456]]

# Create a new optimization model to maximize revenue
model = gb.Model("Variable Pricing Model")

In [123]:
a1, b1 = response[0]
a2, b2 = response[1]

In [124]:
p1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p1")  # Price for Basic version
p2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p2")  # Price for Advanced version

In [125]:
model.setObjective(p1*(a1 - b1*p1) + p2*(a2 - b2*p2), sense=GRB.MAXIMIZE)

In [126]:
model.addConstr(p1 <= p2, "price_ordering_constraint")

# Demand non-negativity constraints
model.addConstr(a1 - b1*p1 >= 0, "demand_nonnegativity_p1")
model.addConstr(a2 - b2*p2 >= 0, "demand_nonnegativity_p2")

<gurobi.Constr *Awaiting Model Update*>

In [127]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xf11202ab
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 4e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     

In [128]:
print("Optimal price for the Basic version (p1):", p1.X)
print("Optimal price for the Advanced version (p2):", p2.X)

Optimal price for the Basic version (p1): 383.84827160836835
Optimal price for the Advanced version (p2): 2296.49891813207


## b

In [129]:
p1 = 0
p2 = 0
step_size = 0.001
stopping_criterion = 1e-6

In [130]:
proj_model = gb.Model("Projected_Gradient_Descent")

In [131]:
p1_var = proj_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p1")
p2_var = proj_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p2")

In [132]:
objective_expr = p1*(a1 - b1*p1_var) + p2*(a2 - b2*p1_var)
proj_model.setObjective(objective_expr, GRB.MAXIMIZE)

In [133]:
constraint1 = proj_model.addConstr(p1_var <= p2_var, name="price_ordering")
constraint2 = proj_model.addConstr(a1 - b1 * p1_var >= 0, name="demand_non_negativity1")
constraint3 = proj_model.addConstr(a2 - b2 * p2_var >= 0, name="demand_non_negativity2")

In [134]:
proj_model.update()
proj_model.Params.OutputFlag = 0  # Suppress Gurobi output

In [135]:
while True:
    # Solve current model
    proj_model.optimize()

    # Extract current prices
    p1_curr = p1_var.X
    p2_curr = p2_var.X

    # Compute gradients
    gradient_p1 = a1 - 2 * b1 * p1_curr
    gradient_p2 = a2 - 2 * b2 * p2_curr

    # Update prices
    p1_next = max(p1_curr + step_size * gradient_p1, 0)
    p2_next = max(p2_curr + step_size * gradient_p2, 0)

    # Update variables
    p1_var.lb = p1_next
    p2_var.lb = p2_next

    # Check stopping criterion
    if max(abs(p1_next - p1_curr), abs(p2_next - p2_curr)) < stopping_criterion:
        break

    # Update current prices
    p1 = p1_next
    p2 = p2_next


In [136]:
print("Optimal prices:")
print("p1:", p1)
print("p2:", p2)

Optimal prices:
p1: 383.84827160837096
p2: 2296.4988578305542


# c

In [137]:
a = [[35234.5457855123, 37790.24083, 35675.33322],
    [37041.38038, 36846.14039, 35827.02375],
    [39414.26632, 35991.95146, 39313.31703]]

b = [[45.8964497063843, 8.22779417263456, 7.5844364095833],
    [9.03316640448659, 4.42786920644331, 2.62906001535909],
    [2.42148391836987, 4.00051240063997, 2.29662237308723]]

capacity = [[80020, 89666, 80638],
            [86740, 84050, 86565],
            [87051, 85156, 87588]]

In [138]:
model_c = gb.Model("LaptopPricing")

In [139]:
prices = {}
for i in range(3):
    for j in range(3):
        prices[i,j] = model_c.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"Price_{i}_{j}")

In [140]:
revenue = gb.quicksum(prices[i,j] * (a[i][j] - b[i][j] * prices[i,j]) for i in range(3) for j in range(3))
model_c.setObjective(revenue, GRB.MAXIMIZE)

In [141]:
for i in range(3):
    for j in range(3):
        model_c.addConstr(a[i][j] - b[i][j] * prices[i,j] >= 0)

In [142]:
for i in range(3):
    for j in range(3):
        model_c.addConstr(a[i][j] - b[i][j] * prices[i,j]<= capacity[i][j])

In [143]:
for i in range(3):
    model_c.addConstr(prices[i, 0] <= prices[i, 1])
    model_c.addConstr(prices[i, 1] <= prices[i, 2])

In [144]:
model_c.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 9 columns and 30 nonzeros
Model fingerprint: 0x8744217b
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [145]:
if model_c.status == GRB.OPTIMAL:
    print("Optimal Revenue: $", round(model_c.objVal, 2))
else:
    print("Optimization could not be completed.")

Optimal Revenue: $ 718382097.68


In [146]:
print("Optimal Prices:")
for i in range(3):
    for j in range(3):
        print(f"Product Line {i+1}, Version {j+1}: ${prices[i, j].X}")

Optimal Prices:
Product Line 1, Version 1: $383.8482716083712
Product Line 1, Version 2: $2296.4989179882505
Product Line 1, Version 3: $2351.8776672154513
Product Line 2, Version 1: $2050.2987945402124
Product Line 2, Version 2: $4160.707856544469
Product Line 2, Version 3: $6813.656504738742
Product Line 3, Version 1: $5870.932809225458
Product Line 3, Version 2: $5870.93280922546
Product Line 3, Version 3: $8558.94236046154


# d

In [147]:
model_d = gb.Model("Pricingconstraints")

In [148]:
prices_d = {}
for i in range(3):
    for j in range(3):
        prices_d[i,j] = model_d.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"Price_{i}_{j}")

In [149]:
revenue = gb.quicksum(prices_d[i,j] * (a[i][j] - b[i][j] * prices_d[i,j]) for i in range(3) for j in range(3))
model_d.setObjective(revenue, GRB.MAXIMIZE)

In [150]:
for i in range(3):
    for j in range(3):
        model_d.addConstr(a[i][j] - b[i][j] * prices_d[i,j] >= 0)

In [151]:
for i in range(3):
    for j in range(3):
        model_d.addConstr(a[i][j] - b[i][j] * prices_d[i,j]<= capacity[i][j])

In [152]:
for i in range(3):
    model_d.addConstr(prices_d[i, 0] <= prices_d[i, 1])
    model_d.addConstr(prices_d[i, 1] <= prices_d[i, 2])

In [153]:
for j in range(3):
    model_d.addConstr(prices_d[0, j] <= prices_d[1, j])
    model_d.addConstr(prices_d[1, j] <= prices_d[2, j])

In [154]:
model_d.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 9 columns and 42 nonzeros
Model fingerprint: 0x5b313137
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual

In [155]:
if model_d.status == GRB.OPTIMAL:
    print("Optimal Revenue: $", round(model_d.objVal, 2))
else:
    print("Optimization could not be completed.")

Optimal Revenue: $ 718382097.68


In [156]:
print("Optimal Prices:")
for i in range(3):
    for j in range(3):
        print(f"Product Line {i+1}, Version {j+1}: ${prices_d[i, j].X}")

Optimal Prices:
Product Line 1, Version 1: $383.84827160837125
Product Line 1, Version 2: $2296.4989179982404
Product Line 1, Version 3: $2351.8776672046133
Product Line 2, Version 1: $2050.2987945402124
Product Line 2, Version 2: $4160.707856544468
Product Line 2, Version 3: $6813.656504738741
Product Line 3, Version 1: $5870.932809225459
Product Line 3, Version 2: $5870.932809225459
Product Line 3, Version 3: $8558.94236046154


# Question 2 

In [157]:
import pandas as pd

In [158]:
m = gb.Model("TrainingCampSelection")

In [159]:
invited = m.addVars(150, vtype=GRB.BINARY, name="Selected Players")

In [160]:
skills =pd.read_csv(r'https://raw.githubusercontent.com/Ninja-Apprentice/Modelling-and-Application/main/BasketballPlayers.csv')
skills

,Number,Position,Ball Handling,Shooting,Rebounding,Defense,Athletic Ability,Toughness,Mental Acuity
0,1,G/F,1,2,3,2,1,2,1
1,2,G/F,1,1,1,2,3,2,3
2,3,G/F,3,1,1,2,3,2,1
3,4,G/F,2,3,2,2,2,1,1
4,5,F/C,1,2,3,3,3,3,2
...,...,...,...,...,...,...,...,...,...
145,146,F/C,2,2,3,1,1,3,3
146,147,G/F,2,3,2,3,2,2,2
147,148,G/F,3,1,2,3,2,3,3
148,149,F/C,1,2,3,1,1,2,2


In [161]:
skills_rating = skills.iloc[:, 2:].values
skills_rating

array([[1, 2, 3, ..., 1, 2, 1],
       [1, 1, 1, ..., 3, 2, 3],
       [3, 1, 1, ..., 3, 2, 1],
       ...,
       [3, 1, 2, ..., 2, 3, 3],
       [1, 2, 3, ..., 1, 2, 2],
       [1, 3, 2, ..., 3, 2, 3]], dtype=int64)

In [162]:
players_positions = skills.iloc[:, 1].values
players_positions

array(['G/F', 'G/F', 'G/F', 'G/F', 'F/C', 'G', 'F/C', 'F/C', 'G', 'G/F',
       'F', 'F/C', 'G/F', 'F', 'G', 'G', 'G', 'F/C', 'F/C', 'G/F', 'F',
       'F/C', 'G', 'F', 'G', 'G', 'G', 'F', 'G', 'G/F', 'G', 'F', 'F',
       'F/C', 'F', 'F/C', 'G/F', 'F/C', 'F', 'G', 'F', 'F', 'G', 'F',
       'F/C', 'F', 'G/F', 'F/C', 'G', 'G/F', 'F', 'G/F', 'F', 'F', 'F',
       'F/C', 'F', 'F', 'F/C', 'G', 'F/C', 'F', 'F', 'G', 'F', 'F', 'G',
       'F', 'G', 'F', 'F', 'F', 'G', 'F/C', 'G/F', 'F', 'F', 'G', 'F',
       'G', 'G/F', 'G/F', 'F/C', 'F', 'G/F', 'G/F', 'G', 'G/F', 'G/F',
       'F', 'G/F', 'G/F', 'F', 'G', 'G/F', 'G/F', 'F', 'F/C', 'G/F',
       'G/F', 'F/C', 'G', 'F', 'G', 'F/C', 'F', 'G', 'F/C', 'G', 'G', 'F',
       'G', 'F/C', 'F/C', 'G', 'F/C', 'F/C', 'G/F', 'G', 'G', 'F', 'F',
       'G', 'F', 'F/C', 'F/C', 'F', 'G/F', 'G', 'F', 'F', 'G/F', 'F',
       'F/C', 'G', 'F/C', 'F', 'G', 'F/C', 'F', 'G', 'F', 'G', 'F', 'F',
       'F/C', 'G/F', 'G/F', 'F/C', 'F/C'], dtype=object)

In [163]:
skills_rating[2]

array([3, 1, 1, 2, 3, 2, 1], dtype=int64)

In [164]:
m.setObjective(quicksum(skills_rating[i][j] * invited[i] for i in range(150) for j in range(7)), GRB.MAXIMIZE)

In [165]:
m.addConstr(quicksum(invited[i] for i in range(150)) == 21)

<gurobi.Constr *Awaiting Model Update*>

In [166]:
m.addConstr(quicksum(invited[i] for i in range(150) if players_positions[i] in ['G', 'G/F']) >= 0.3 * 21)
m.addConstr(quicksum(invited[i] for i in range(150) if players_positions[i] in ['F', 'C', 'F/C']) >= 0.4 * 21)

<gurobi.Constr *Awaiting Model Update*>

In [167]:
for j in range(7):
    m.addConstr((1 / 21) * quicksum(skills_rating[i][j] * invited[i] for i in range(150)) >= 2.05)
    

In [168]:
m.addConstr(gb.quicksum(invited[i] for i in range(19, 24)) <= 5 + 5 * gb.quicksum(1 - invited[j] for j in range(71, 78)))

for i in range(104, 114):
    for k in range(44, 49):
        for l in range(64, 69):
            m.addConstr(2*invited[i] <= invited[k]+invited[l])

In [169]:
for j in range(15):
    m.addConstr(quicksum(invited[i] for i in range(j*10 , (j+1)*10)) >= 1)

In [170]:
for j in range(15):
  lower_bound = j * 10 
  upper_bound = (j + 1) * 10
  print(f"Range for invited[{j}]: ({lower_bound}, {upper_bound})")

Range for invited[0]: (0, 10)
Range for invited[1]: (10, 20)
Range for invited[2]: (20, 30)
Range for invited[3]: (30, 40)
Range for invited[4]: (40, 50)
Range for invited[5]: (50, 60)
Range for invited[6]: (60, 70)
Range for invited[7]: (70, 80)
Range for invited[8]: (80, 90)
Range for invited[9]: (90, 100)
Range for invited[10]: (100, 110)
Range for invited[11]: (110, 120)
Range for invited[12]: (120, 130)
Range for invited[13]: (130, 140)
Range for invited[14]: (140, 150)


In [171]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 276 rows, 150 columns and 2262 nonzeros
Model fingerprint: 0xef44a8b7
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 5e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 275 rows, 150 columns, 1911 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.580000e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

In [172]:
selected_players = []
print("Selected players:")
for i in range(150):
    if invited[i].X > 0.5:
        selected_players.append(i)
        print(f"Player {i+1}")
        
print("\nTotal skill level of selected players:", m.objVal)

Selected players:
Player 5
Player 7
Player 11
Player 26
Player 37
Player 41
Player 56
Player 70
Player 74
Player 76
Player 90
Player 95
Player 104
Player 118
Player 128
Player 132
Player 133
Player 134
Player 141
Player 144
Player 148

Total skill level of selected players: 358.0


# g

In [173]:
num_guards = sum(1 for i in selected_players if 'G' in players_positions[i])
print("Number of guards invited:", num_guards)

Number of guards invited: 9


# h

In [178]:
m_f = gb.Model("Player_Selection")
x = m_f.addVars(150, vtype=GRB.BINARY, name="x")
m_f.setObjective(quicksum(skills_rating[i][j] * x[i] for i in range(150) for j in range(7)), GRB.MAXIMIZE)
    
m_f.addConstr(quicksum(x[i] for i in range(150) if players_positions[i] in ['G', 'G/F']) >= 0.3 * 21)
m_f.addConstr(quicksum(x[i] for i in range(150) if players_positions[i] in ['F', 'C', 'F/C']) >= 0.4 * 21)
    
for j in range(7):
    m_f.addConstr((1 / 21) * quicksum(skills_rating[i][j] * x[i] for i in range(150)) >= 2.05)
    
m_f.addConstr(gb.quicksum(x[i] for i in range(19, 24)) <= 5 + 5 * gb.quicksum(1 - x[j] for j in range(71, 78)))

for i in range(104, 114):
  for k in range(44, 49):
    for l in range(64, 69):
        m_f.addConstr(2*x[i] <= x[k]+x[l])
    
for j in range(15):
    m_f.addConstr(quicksum(x[i] for i in range(j*10 , (j+1)*10)) >= 1)
    
# Solve the model iteratively
num_invitations = 150
while True:
    # Set the maximum number of solutions to 2
    m_f.setParam('PoolSolutions', 2)
    
    # Set the maximum time limit for optimization
    m_f.setParam('TimeLimit', 600)  # 10 minutes
    
    # Optimize the model
    m_f.optimize()
    
    # If the model is infeasible, print the smallest number of invitations and break the loop
    if m_f.status == gb.GRB.INFEASIBLE:
        print("Smallest number of invitations:", num_invitations)
        break
    
    # Decrease the number of invitations by 1 for the next iteration
    num_invitations -= 1
    
    # Update the constraint for the new number of invitations
    m_f.addConstr(gb.quicksum(x[i] for i in range(150)) <= num_invitations)


Set parameter PoolSolutions to value 2
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 275 rows, 150 columns and 2112 nonzeros
Model fingerprint: 0x577e7c95
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 5e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 2133.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2133 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.133000000000e+03, best bound 2.133000000000e+03, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v1

### The constraint "m.addConstr(quicksum(invited[i] for i in range(150)) == 21)" cannot be satisfied

# i

In [ ]:
for i in range(150):
    m.addConstr(quicksum(skills_rating[i][j] * invited[i] for j in range(7)) >= 12 * invited[i])

In [ ]:
quicksum(skills_rating[i][j] * invited[i] for j in range(7)) >= 12 * invited[i]

<gurobi.TempConstr: Selected Players[149] + 3.0 Selected Players[149] + 2.0 Selected Players[149] + Selected Players[149] + 3.0 Selected Players[149] + 2.0 Selected Players[149] + 3.0 Selected Players[149] >= 12.0 Selected Players[149]>